In [1]:
import bs4
from bs4 import BeautifulSoup
import statsmodels as sm
from tqdm import tqdm
import urllib2, requests
import pandas as pd
import simplejson
import re
import hashlib
%matplotlib inline

In [2]:
# Create a list of urls that will be used to scrape the data. In each url there are 20 offers for buying/renting apartments:
links = []
for i in range(4500):
    a = i*21
    if a == 0:
        b = a
    else:
        b = a -21
    lin = "http://www.squarefoot.com.hk/search/?type=rent;from=" + str(b) + ";from=" + str(a)
    links.append(lin)
#--------------------------------------------------------------------------------------------------------------    
#Create a hash code for each one of the url -> Possible use in the future to make sure each url is unique:

cod = []

for i in links:
    hash_obj = hashlib.md5(i)
    cod.append(hash_obj.hexdigest())
#--------------------------------------------------------------------------------------------------------------    
#Save dictionary of hashcode : urls:
df = pd.DataFrame({"cod" : cod, "url" : links})
df.to_csv("../datasets/URL_COD")
#--------------------------------------------------------------------------------------------------------------    
#Create a dict of hashcode : urls, iterate dict values, opening url and saving them in a html file located in tons folder:
urls = dict(zip(cod, links))
for i, j in  tqdm(urls.iteritems()):
    html_content = urllib2.urlopen(j)
    content = html_content.read()
    folder = "tons"
 
    filename = folder + '/' + i + ".html" 
    with file(filename, "wt") as fo:
        fo.write(content)
#--------------------------------------------------------------------------------------------------------------    
#Create a list of all the html files saved in tons folder:
newlist1 = []
for i in cod:
    var = "file:///C:/Users/alexrebull/.babun/cygwin/home/alexrebull/DSI-HK-1/capstone/Assets/scraping/tons/" + i + ".html"
    newlist1.append(var)
#--------------------------------------------------------------------------------------------------------------    
#Extract all the information from each of the urls:    
name = []
proptype = []
bedroom = []
bathroom = []
gross_ar = []
salar_ar = []
description = []
mortgage = []
rs = []
ref = []
sell = []
rent = []
#Lists that need cleaning---
gross_ar1 = []
sal_ar1 = []
xd = []
price = []
ref1=[]
mortgage1 = []
#------
for i in tqdm(newlist1):
    contents = BeautifulSoup(urllib2.urlopen(i), "lxml")
    contentsElements = contents.find_all('div', attrs={'class' : 'standard'})
    for elem in contentsElements:
        name.append(elem.find("a").get_text())
        proptype.append(elem.find('span', attrs={'class': 'propertyType'}).getText())
    contentsElements1 = contents.find_all('div', attrs={'class' : 'price'})
    for elem in contentsElements1:
        #Extracting apartment price:
        price.append(elem.getText())
    contentsElements2 = contents.find_all('li', attrs={'class' : 'first'})
    for elem in contentsElements2:
        if elem.find("span") != None:
            #Extracting number of bedrooms, if there is no info -> number of bedrooms = None
            bedroom.append(elem.find("span").get_text())
        else:
            bedroom.append(None)
    contentsElements3 = contents.find_all('img', attrs={'class' : 'Bathroom(s)'})
    for elem in contentsElements3:
        print elem.find("span")
    contentsElements4 = contents.find_all("h3")
    for elem in contentsElements4:
        #Extracting the gross/saleable area:
        if len(elem.getText()) > 25:
            gross_ar1.append(elem.getText().split("|")[0])
            sal_ar1.append(elem.getText().split("|")[1])
    contentsElements5 = contents.find_all('div', attrs={'class' : 'desc'})
    for elem in contentsElements5:
        #Extracting the description of the apartment:
        description.append(elem.getText().encode('utf-8' , 'ignore'))
    contentsElements6 = contents.find_all('div', attrs={'class' : 'mortgage'})
    for elem in contentsElements6:
        #Extracting the mortgage:
        if elem.find("a") != None:
            mortgage1.append(elem.find("a").get_text())
        else:
            mortgage1.append(None)
    contentsElements7 = contents.find_all('div', attrs={'class' : 'buttons'})
    for elem in contentsElements7:
        x = elem.findAll("a")
        #Extracting the reference number:
        for p in x:
            ref1.append(p["href"])
    for i in contentsElements:
        if i.find("div", attrs={"class" : "imglogo"}):
            lel = i.findAll("a")
            #Extracting the Real Estate Agency:
            for u in lel:
                hue = u.findAll("img", attrs={"width" : "160"})
                for t in hue:
                    rs.append(t["alt"])
                    
                
        else:
            rs.append(None)
            
for i in sal_ar1:
    xd.append(i.split("\r\n\t\t\t")[1])
for i in gross_ar1:
    gross_ar.append(i.split()[2])
for i in xd:
    salar_ar.append(i.split()[0])
for i in price:
    if "million" in i:
        sell.append(i.split()[2])   
    else: 
        sell.append(None)
for i in price:
    if "million" in i:
        rent.append(i.split()[6]) 
    else: 
        rent.append(i.split()[2])
for i in ref1:
    ref.append(i.split("/")[2])
for i in mortgage1:
    if i != None:
        mortgage.append(i.split()[5])
    else:
        mortgage.append(None)
        
#Create a csv file saving the reference number:
df = pd.DataFrame(ref)
df.to_csv("../datasets/refer")

#Create a dataframe and save it as a csv file called bigdf:
df = pd.DataFrame({"name": name, "proptype" : proptype, "bedroom": bedroom, "gross_ar" : gross_ar, "salar_ar" : salar_ar, 
                   "description" : description, "mortgage" : mortgage, "real state" : rs, "ref" : ref, "sell" : sell, "rent": rent })

df = df[["ref", "name", "sell", "rent", "gross_ar", "salar_ar", "real state", "mortgage", "bedroom", "proptype", "description"]]

df= df.drop_duplicates(subset = "ref")

df.to_csv("../datasets/bigdf")

#----------------------------------------------------------------------------------------------------------------------------------